## GR5245 Homework 3

(a) Download the TensorFlow flowers dataset from tensorflow_datasets. Take the first 80% as 
the training set and the remaining 20% as the validation set. 

In [80]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


In [81]:
(x_train, x_valid), info = tfds.load('tf_flowers',
                                     split=['train[:80%]',
                                            'train[80%:]'],
                                     with_info=True,
                                     as_supervised=True)


In [82]:
b = np.array([x[1].numpy() for x in x_train])

(b) Write a preprocessing function to resize the images (both in training and validation sets) to have 
64x64 pixel values and rescale all pixel values to have values between 0 and 1. Build a data pipeline 
which will generate mini-batches of 32 samples randomly selected from a buffer of 1000 items. 

In [83]:
def preprocess(image, label):
    image = tf.image.resize(image, size=(64, 64))/255.0
    return image, label


In [84]:
train_ds = x_train.map(preprocess).shuffle(
    1000, reshuffle_each_iteration=True).batch(32)
valid_ds = x_valid.map(preprocess).batch(32)


(c) Create a Keras model as shown in the table below. Compile the model using SGD with a constant 
learning rate of 0.001, sparse categorical entropy as loss function and sparse categorical accuracy as 
the metric. Train the model for 20 epochs using the data pipeline prepared in (b). Keep the training 
history for later use.  

In [85]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(2, 2, padding="valid"),
    tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(2, 2, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])


In [86]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [87]:
history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20
92/92 [==============================] - 3s 31ms/step - loss: 1.6048 - sparse_categorical_accuracy: 0.2268 - val_loss: 1.5910 - val_sparse_categorical_accuracy: 0.3038
Epoch 2/20
92/92 [==============================] - 3s 28ms/step - loss: 1.5850 - sparse_categorical_accuracy: 0.2912 - val_loss: 1.5729 - val_sparse_categorical_accuracy: 0.3433
Epoch 3/20
92/92 [==============================] - 3s 28ms/step - loss: 1.5652 - sparse_categorical_accuracy: 0.3433 - val_loss: 1.5515 - val_sparse_categorical_accuracy: 0.3787
Epoch 4/20
92/92 [==============================] - 3s 28ms/step - loss: 1.5437 - sparse_categorical_accuracy: 0.3597 - val_loss: 1.5360 - val_sparse_categorical_accuracy: 0.3474
Epoch 5/20
92/92 [==============================] - 2s 26ms/step - loss: 1.5202 - sparse_categorical_accuracy: 0.3709 - val_loss: 1.5100 - val_sparse_categorical_accuracy: 0.4019
Epoch 6/20
92/92 [==============================] - 2s 27ms/step - loss: 1.4956 - sparse_categorical_accu

(d) Create a new model by adding batch normalization layers as shown below. Compile and train the 
model using the same hyper parameters as in (c). Keep the training history for later use. 

In [88]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 2, padding="valid"),

    tf.keras.layers.Conv2D(64, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 2, padding="valid"),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(5, activation="softmax")
])


In [89]:
model2.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [90]:
history2 = model2.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20
92/92 [==============================] - 3s 33ms/step - loss: 1.4773 - sparse_categorical_accuracy: 0.4009 - val_loss: 1.8348 - val_sparse_categorical_accuracy: 0.2166
Epoch 2/20
92/92 [==============================] - 3s 34ms/step - loss: 1.1789 - sparse_categorical_accuracy: 0.5364 - val_loss: 2.0953 - val_sparse_categorical_accuracy: 0.2166
Epoch 3/20
92/92 [==============================] - 3s 36ms/step - loss: 1.0557 - sparse_categorical_accuracy: 0.5896 - val_loss: 1.9162 - val_sparse_categorical_accuracy: 0.2398
Epoch 4/20
92/92 [==============================] - 3s 36ms/step - loss: 0.9832 - sparse_categorical_accuracy: 0.6298 - val_loss: 1.5296 - val_sparse_categorical_accuracy: 0.3283
Epoch 5/20
92/92 [==============================] - 3s 37ms/step - loss: 0.9083 - sparse_categorical_accuracy: 0.6666 - val_loss: 1.2385 - val_sparse_categorical_accuracy: 0.4959
Epoch 6/20
92/92 [==============================] - 3s 37ms/step - loss: 0.8489 - sparse_categorical_accu

(e) Write a data augmentation function that either flips an image vertically or adjust the contrast factor 
to 1.5 (with equal probability of 50%). Use this function to create a new dataset from the original 
training set in (a). Merge the new dataset and the original training set to form an enlarged training 
dataset. 

In [91]:
def augmentation(image, label):
    if np.random.choice((False, True)):
        image = tf.image.flip_up_down(image)
    else:
        image = tf.image.adjust_contrast(image, 1.5)
    return image, label

In [92]:
train_aug = x_train.map(augmentation)

In [93]:
train_large = x_train.concatenate(train_aug)

(f) Similar to (b), build a data pipeline for the enlarged training set created in (e).  

In [94]:
train_ds_aug = train_large.map(preprocess).shuffle(
    1000, reshuffle_each_iteration=True).batch(32)


(g) Create a new model with same architecture and parameters as the model in (d). Compile and train 
the model for 20 epochs using the data pipeline prepared in (f).

In [95]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 2, padding="valid"),

    tf.keras.layers.Conv2D(64, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 2, padding="valid"),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(5, activation="softmax")
])

In [96]:
model3.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [97]:
history3 = model3.fit(train_ds_aug,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20
184/184 [==============================] - 7s 34ms/step - loss: 1.3577 - sparse_categorical_accuracy: 0.4564 - val_loss: 2.1273 - val_sparse_categorical_accuracy: 0.2166
Epoch 2/20
184/184 [==============================] - 6s 33ms/step - loss: 1.0477 - sparse_categorical_accuracy: 0.6000 - val_loss: 1.4448 - val_sparse_categorical_accuracy: 0.3583
Epoch 3/20
184/184 [==============================] - 6s 34ms/step - loss: 0.9559 - sparse_categorical_accuracy: 0.6381 - val_loss: 1.0386 - val_sparse_categorical_accuracy: 0.5940
Epoch 4/20
184/184 [==============================] - 6s 34ms/step - loss: 0.8907 - sparse_categorical_accuracy: 0.6698 - val_loss: 0.9883 - val_sparse_categorical_accuracy: 0.6090
Epoch 5/20
184/184 [==============================] - 6s 35ms/step - loss: 0.8306 - sparse_categorical_accuracy: 0.6965 - val_loss: 0.9515 - val_sparse_categorical_accuracy: 0.6294
Epoch 6/20
184/184 [==============================] - 7s 36ms/step - loss: 0.7797 - sparse_cate